## Register a metadata schema

The registration of the metadata schema is usually done by the administrator of the system. The metadata schema is defined in a JSON file, which is then registered in the system. This registration is not usually done as part of running tests or capturing data

In [ ]:
from ni.datastore import Client

client = Client()
with open("sample_schema.json", "r", encoding="utf-8") as f:
    contents = f.read()
schema_id = client.register_schema(contents)
print(f"registered schema id: {schema_id}")

## Create the test result metadata

Now that a schema has been registered we can create our metadata objects with the expected metadata.  Note, however, the creation of each metadata object must include the required values from the schema.

In [ ]:
import grpc
from ni.datastore import TestResult
from ni.measurements.metadata.v1.metadata_store_pb2 import Operator, TestStation, SoftwareItem, ExtensionValue

# Invalid creation of operator - no badge_number provided
operator = Operator(operator_name="James Bowery", schema_id=schema_id)
try:
    operator_id = client.create_operator(operator)
except grpc.RpcError as e:
    print("Failed to create operator:")
    print(f"  {e.details()}")

# Add the required attribute to the operator object
operator.extensions["badge_number"].CopyFrom(ExtensionValue(string_value="emp-128256"))
operator_id = client.create_operator(operator)

# Invalid creation of test station - location is invalid
test_station = TestStation(test_station_name="TestStation_12", schema_id=schema_id)
test_station.extensions["location"].CopyFrom(ExtensionValue(string_value="Texas"))
try:
    test_station_id = client.create_test_station(test_station)
except grpc.RpcError as e:
    print("Failed to create test station:")
    print(f"  {e.details()}")

# Fix the location
test_station.extensions["location"].CopyFrom(ExtensionValue(string_value="USA"))
test_station_id = client.create_test_station(test_station)

# Invalid creation of software item -  software license is invalid (not matching the pattern defined in the schema)
software_item = SoftwareItem(product="Windows", version="10.0.19044", schema_id=schema_id)
software_item.extensions["license"].CopyFrom(ExtensionValue(string_value="enterprise_LIC"))
try:
    software_item_id = client.create_software_item(software_item)
except grpc.RpcError as e:
    print("Failed to create software item:")
    print(f"  {e.details()}")

# Fix the software license to create the software item
software_item.extensions["license"].CopyFrom(ExtensionValue(string_value="LIC_enterprise"))
software_item_id = client.create_software_item(software_item)
    
test_result = TestResult(
    test_result_name="sample publish test result",
    operator_id=operator_id,
    test_station_id=test_station_id,
    schema_id=schema_id,
    software_item_ids=[software_item_id])
test_result.extensions["session_file_path"] = ExtensionValue(string_value="C:\\my_test_description.xlsx")


## Create the test result object

Now that we have the test result metadata, we can create the test result object. Note below that schema validation can fail if your metadata does not match what's specified in the schema.

In [ ]:
test_result_id = client.create_test_result(test_result)
print(f"created test result id: {test_result_id}")

## Publish the data

Now that we have a valid test result and schema, we can publish our data

In [ ]:
from datetime import timedelta, timezone
import hightime as ht
from nitypes.waveform import AnalogWaveform
from nitypes.waveform import Timing
import numpy as np
from ni.datastore import Step

name = "data publish sample"
waveform = AnalogWaveform(
    sample_count=3,
    raw_data=np.array([1.0, 2.0, 3.0]),
    timing=Timing.create_with_regular_interval(
        timedelta(seconds=1e-3),
        ht.datetime.now(timezone.utc)
    )
)
step = Step(step_name="Initial step", test_result_id=test_result_id)
step_id = client.create_step(step)
published_measurement = client.publish_measurement(
    measurement_name=name,
    value=waveform,
    step_id=step_id,
)

print(f"The published measurement id is {published_measurement.published_measurement_id}")

## Query the data

Now that we've got data in the database, we can query for it using our OData query api.

In [ ]:
from nitypes.waveform import AnalogWaveform

published_measurements = client.query_measurements(odata_query=f"$filter=id eq {published_measurement.published_measurement_id}")
found_measurement = next(iter(published_measurements), None)

if found_measurement is not None:
    test_result = client.get_test_result(found_measurement.test_result_id)
    operator = client.get_operator(test_result.operator_id)
    # badge_number is a custom attribute in the schema
    # It's not a standard attribute of the operator object
    badge_number = operator.extensions["badge_number"].string_value
    print(f"operator {operator.operator_name}'s badge number is: {badge_number}")

    waveform = client.read_data(found_measurement, expected_type=AnalogWaveform)
    print(f"published data is: {waveform.raw_data}")